In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler,PowerTransformer
import joblib
import pickle

In [2]:
df_ml = pd.read_csv("/home/kiran/medicare/data/output/processed_data.csv")
print("Processed data loaded for prediction")

Processed data loaded for prediction


In [3]:
df_ml.head(1)

,Unnamed: 0,NPI,City,State,First_name,last_org_name,Speciality,sum_Tot_Drug_Cst,avg_Tot_Drug_Cst,max_Tot_Drug_Cst,...,Diazepam_Tot_Clms,Diazepam_Tot_Day_Suply,"Pen Needle, Diabetic_Tot_Drug_Cst","Pen Needle, Diabetic_Tot_Day_Suply",Tramadol Hcl_Tot_Drug_Cst,Tramadol Hcl_Tot_Clms,Tramadol Hcl_Tot_Day_Suply,claim_max-mean,supply_max-mean,drug_max-mean
0,0,1003000126,Bethesda,MD,Ardalan,Enkeshafi,Internal Medicine,5979.070107,747.383763,4792.850098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,471.875,4045.466334


In [4]:
#df_ml.drop(["Unnamed: 0","State","NPI"],axis =1,inplace= True)
df_ml.drop(["Unnamed: 0","State","City","First_name","last_org_name","Unnamed: 0_y"],axis =1,inplace= True)

df_ml = df_ml.round(2)

In [5]:
df_ml.head(1)

,NPI,Speciality,sum_Tot_Drug_Cst,avg_Tot_Drug_Cst,max_Tot_Drug_Cst,sum_Total_claims,avg_Total_claims,max_Total_claims,sum_Tot_Day_Suply,avg_Tot_Day_Suply,...,Diazepam_Tot_Clms,Diazepam_Tot_Day_Suply,"Pen Needle, Diabetic_Tot_Drug_Cst","Pen Needle, Diabetic_Tot_Day_Suply",Tramadol Hcl_Tot_Drug_Cst,Tramadol Hcl_Tot_Clms,Tramadol Hcl_Tot_Day_Suply,claim_max-mean,supply_max-mean,drug_max-mean
0,1003000126,Internal Medicine,5979.07,747.38,4792.85,124.0,15.5,30.0,3721.0,465.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,471.88,4045.47


In [6]:
df_ml.isna().sum()

NPI                           0
Speciality                    0
sum_Tot_Drug_Cst              0
avg_Tot_Drug_Cst              0
max_Tot_Drug_Cst              0
                             ..
Tramadol Hcl_Tot_Clms         0
Tramadol Hcl_Tot_Day_Suply    0
claim_max-mean                0
supply_max-mean               0
drug_max-mean                 0
Length: 120, dtype: int64

### Scaling

In [7]:
# df_ml.columns.values

In [8]:
scale_features = df_ml.columns.to_list()[2:15]
# scale_features.append("Total_payments")
scale_features

['sum_Tot_Drug_Cst',
 'avg_Tot_Drug_Cst',
 'max_Tot_Drug_Cst',
 'sum_Total_claims',
 'avg_Total_claims',
 'max_Total_claims',
 'sum_Tot_Day_Suply',
 'avg_Tot_Day_Suply',
 'max_Tot_Day_Suply',
 'sum_Tot_30day_Fills',
 'avg_Tot_30day_Fills',
 'max_Tot_30day_Fills',
 'Total_payments']

 ### Power transformer scaler: 

In [9]:
pt = PowerTransformer()

In [10]:
df_pscale = df_ml.copy()
df_pscale[scale_features] = pd.DataFrame(
    pt.fit_transform(df_pscale.loc[:,scale_features]), columns=scale_features
)

print("Features scaled by power transformer")

Features scaled by power transformer


### Scaler assignment

In [11]:
#copying scaled output to new variable for further processiing

df_scale = df_pscale.copy()

In [12]:
#FFilled missing value with most occuring speciality

df_scale["Speciality"].fillna(df_scale.Speciality.mode,inplace = True)
df_scale["Speciality"].isnull().sum()

0

In [13]:
# # make list with top 30 variables
# top_30 = [x for x in df_scale.Speciality.value_counts().sort_values(ascending=False).head(30).index]
# #top_30.append('anesthesiology')
# top_30

In [14]:
top_30 = [line.rstrip() for line in open('/home/kiran/medicare/data/top30.txt')]
top_30

['nurse practitioner',
 'internal medicine',
 'family practice',
 'dentist',
 'physician assistant',
 'emergency medicine',
 'student in an organized health care education/training program',
 'optometry',
 'obstetrics & gynecology',
 'pharmacist',
 'psychiatry',
 'cardiology',
 'orthopedic surgery',
 'ophthalmology',
 'general surgery',
 'gastroenterology',
 'neurology',
 'podiatry',
 'dermatology',
 'hospitalist',
 'psychiatry & neurology',
 'urology',
 'pulmonary disease',
 'otolaryngology',
 'nephrology',
 'hematology-oncology',
 'general practice',
 'physical medicine and rehabilitation',
 'endocrinology',
 'oral surgery (dentist only)']

In [15]:
# for all categorical variables we selected
def top_x(df2,variable,top_x_labels):
    
    for label in top_x_labels:
        df2[variable+'_'+label] = np.where(df_scale[variable]==label,1,0)

In [16]:
#encode Nighborhood into the 10 most frequent categories
top_x(df_scale,'Speciality',top_30)
# display data
df_scale.tail(2)

print("One hot encoding done")

One hot encoding done


In [17]:
df_scale.drop("Speciality",axis=1,inplace = True)

In [18]:
df_scale.head(1)

,NPI,sum_Tot_Drug_Cst,avg_Tot_Drug_Cst,max_Tot_Drug_Cst,sum_Total_claims,avg_Total_claims,max_Total_claims,sum_Tot_Day_Suply,avg_Tot_Day_Suply,max_Tot_Day_Suply,...,Speciality_psychiatry & neurology,Speciality_urology,Speciality_pulmonary disease,Speciality_otolaryngology,Speciality_nephrology,Speciality_hematology-oncology,Speciality_general practice,Speciality_physical medicine and rehabilitation,Speciality_endocrinology,Speciality_oral surgery (dentist only)
0,1003000126,-0.146622,-0.044038,0.093529,-0.437908,-1.283649,-0.774999,-0.277338,-0.445939,-0.433071,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#Total columns 226 , fraud column at the end

df_scale.columns

Index(['NPI', 'sum_Tot_Drug_Cst', 'avg_Tot_Drug_Cst', 'max_Tot_Drug_Cst',
       'sum_Total_claims', 'avg_Total_claims', 'max_Total_claims',
       'sum_Tot_Day_Suply', 'avg_Tot_Day_Suply', 'max_Tot_Day_Suply',
       ...
       'Speciality_psychiatry & neurology', 'Speciality_urology',
       'Speciality_pulmonary disease', 'Speciality_otolaryngology',
       'Speciality_nephrology', 'Speciality_hematology-oncology',
       'Speciality_general practice',
       'Speciality_physical medicine and rehabilitation',
       'Speciality_endocrinology', 'Speciality_oral surgery (dentist only)'],
      dtype='object', length=149)

In [20]:
npi_ = pd.DataFrame(df_scale['NPI'])

In [21]:
npi_.head()

,NPI
0,1003000126
1,1003000423
2,1003001785
3,1003002049
4,1003002890


In [22]:
npi_.shape

(886, 1)

In [23]:
df_scale.drop("NPI",axis =1,inplace=True)

In [24]:
# model_ = joblib.load("/home/kiran/medicare/src_deploy/model/rf.joblib")
#result = model.predict_and_evaluate(X_test, Y_test)
#print(result)

In [25]:
# joblib.dump(model_,"/home/kiran/medicare/src_deploy/model/rf_1.1.joblib")

In [26]:
# with open('/home/kiran/medicare/src_deploy/model/model_pkl', 'wb') as files:
#     pickle.dump(model_, files)

In [27]:
with open('/home/kiran/medicare/src_deploy/model/model_pkl' , 'rb') as f:
    model_ = pickle.load(f)

In [28]:
#result = model_.score(X_test, y_test)
#print(result)

In [29]:
#res=predict_and_evaluate(model_,X_test,y_test)

In [30]:
g_pred=model_.predict(df_scale)

print("Predictions done")

Predictions done


/home/kiran/project/venv/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [31]:
# g_pred

In [32]:
npi_["pred"] = g_pred

In [33]:
npi_.head()

,NPI,pred
0,1003000126,0
1,1003000423,0
2,1003001785,0
3,1003002049,0
4,1003002890,0


In [34]:
npi_.pred.value_counts()

0    654
1    232
Name: pred, dtype: int64

In [35]:
npi_ = npi_[npi_.pred == 1]

NameError: name 'a' is not defined

In [39]:
np.savetxt(r"/home/kiran/medicare/data/output/np.txt", npi_.NPI.values, fmt='%d')

In [ ]:
npi_.NPI.to_csv("/home/kiran/medicare/data/output/NPI_FNF.csv")

print("NPI of fraud practioner saved")